In [ ]:
%matplotlib inline
#!pip install netCDF4
import numpy as np
import sklearn as skl
from netCDF4 import Dataset
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Random Forest Regression for Scattering
So the first concrete example using radar data we will look at is attempting to use random forests for regression. 

Our goal is to train a random forest regression to predict the DSD based on scattered moments. Ahead of time we have taken data representing drop size distributions that measure the distribution of different raindrop sizes in clouds and done the electromagnetic scattering calculations to turn them into equivalent radar measurements. We've then written these out to a file to make it easier to process here. 



!wget https://github.com/josephhardinee/weather_radar_ml_course/raw/master/x_scattered_dsds.nc


This file contains several fields. The first three are $D_0$, $N_W$ and $\mu$. These represent the parameterization of a 3 parameter gamma distribution. If you are not familiar with DSDs, think of $D_0$ as telling you the median size of the drops, $N_W$ as roughly the number of drops, and $\mu$ as the shape of the distribution. 

From each of these distributions, we can calculate equivalent radar measurements using T-Matrix scattering theory. This is done using two libraries (PyDSD and PyTMatrix). We have done these calculations for a hypothetical X band radar with a specific configuration. We now want to see if we can use radar measurements to recover the DSD estimates. Note the scattering is a highly nonlinear operation. 

In [ ]:
# Let's read in the data.

dset = Dataset('x_scattered_dsds.nc')
d0 = dset['D0'][:]
nw = dset['Nw'][:]
mu = dset['mu'][:]
zh = dset['Zh'][:]
zdr = dset['Zdr'][:]
kdp = dset['Kdp'][:]

First we get our data in a form more expected by scikit learn. 

In [ ]:
X = np.array((zh, zdr, kdp)).T
y = np.array((d0, nw, mu)).T

X_train = X[0:35000]
y_train = y[0:35000]

X_test = X[35000:]
y_test = y[35000:]

In [ ]:
X.shape

So first we can start by visualizing our data a little bit. 

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.subplot(1,3,1)
plt.hist(zh, log=True)
plt.xlabel('Zh')
plt.ylabel('Counts')

plt.subplot(1,3,2)
plt.hist(zdr)
plt.xlabel('Zdr')
plt.ylabel('Counts')

plt.subplot(1,3,3)
plt.hist(kdp, log=True)
plt.xlabel('Kdp')
plt.ylabel('Counts')

plt.tight_layout()


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.hist(d0)
plt.xlabel('$D_0$')
plt.ylabel('Counts')

plt.subplot(1,3,2)
plt.hist(nw)
plt.xlabel('$N_W$')
plt.ylabel('Counts')

plt.subplot(1,3,3)
plt.hist(mu)
plt.xlabel('$\mu$')
plt.ylabel('Counts')

plt.tight_layout()

Finally let's take a look at some of the relations between these variables. 

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.hist2d(d0, zdr, bins=100);
plt.xlabel('D0')
plt.ylabel('Zdr')

plt.subplot(1,3,2)
plt.hist2d(d0, zh, bins=100);
plt.xlabel('D0')
plt.ylabel('Zh')

plt.subplot(1,3,3)
plt.hist2d(d0, kdp, bins=100);
plt.xlabel('D0')
plt.ylabel('Kdp')

Exercises:
1. What appears to be the per variable variability for different regions? 
2. What is wrong with the 3rd plot? How can we fix this? 
3. Plot some of the other relationships

Okay now that we understand our data a little bit better, we would start by trying to fit some simple relationships. In general we tend to use power law type fits for most of these. For instance:

$$ D_0 = \alpha Z_{DR}^b $$

These ...work. More so for some variables than others. We can see in our images though that even for D0, this fails to account for the Mie variability in the scattering. Can we do better by using machine learning and a more advanced estimator? It turns out..remarkably yes. 

We will start by training an ensemble classifier called Random Forest. This trains a series of estimation trees, then merges the output. As this is part of scikit learn, we can just pass in our normal style of data (x_train, y_train). 

In [ ]:
regr = RandomForestRegressor( random_state=0, n_estimators=2, max_depth=3)
regr.fit(X_train, y_train)
print(regr.feature_importances_)


In [ ]:
regr.predict(X_test[0:5])

In [ ]:
y_test[0:5]

Let's see how we did!

In [ ]:
y_predict = regr.predict(X_test)

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
plt.scatter(y_predict[:,0], y_test[:,0])
plt.xlabel('Actual D0')
plt.ylabel('Predicted D0')

plt.subplot(1,3,2)
plt.scatter(y_predict[:,1], y_test[:,1])

plt.subplot(1,3,3)
plt.scatter(y_predict[:,2], y_test[:,2])

That is...not good. Well above we forced these trees to be very shallow, and not many of them. We can try increasing the size of our tree to get a little bit more learning power. 

Try increasing the size of the tree some. 

### Exercises:
1. Try changing the size of the tree and rerunning the next two cells below. What works best for you?
2. Why do we avoid making our tree too big? 
3. What other parameters might we want to change. the documentation is at: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [ ]:
n_estimators = 2
max_depth=3
regr = RandomForestRegressor( random_state=0, n_estimators=n_estimators, max_depth=max_depth)
regr.fit(X_train, y_train)
print(regr.feature_importances_)

y_predict = regr.predict(X_test)
print(regr.feature_importances_)


In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
plt.scatter(y_predict[:,0], y_test[:,0])
plt.xlabel('Actual D0')
plt.ylabel('Predicted D0')

plt.subplot(1,3,2)
plt.scatter(y_predict[:,1], y_test[:,1])

plt.subplot(1,3,3)
plt.scatter(y_predict[:,2], y_test[:,2])

Okay that is __MUCH__ better. Let's introduce a loss metric to figure out just how good it is. 

In [ ]:
def MSE(yHat, y):
    return np.sum((yHat - y)**2) / y.size

In [ ]:
D0_loss = MSE(y_predict[:,0], y_test[:,0])
Nw_loss = MSE(y_predict[:,1], y_test[:,1])
mu_loss = MSE(y_predict[:,2], y_test[:,2])
print(D0_loss, Nw_loss, mu_loss)

So we see the random forest was able to very accurately capture D0 and Nw, but did a mediocre job on $\mu$. This is not completely unsurprising as $\mu$ is a very hard variable to capture. This performance is __very__ good. 

## Exploration
So now lets try a few things to see if we can do any better. 
1. Can we visualize why $\mu$ is worse? 
2. Does this imply anything about $\mu$?

# Gradient Boosting Trees Regression
Let's try another algorithm and see if we can do better. 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
est_D0 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,  random_state=0, loss='ls')\
    .fit(X_train, y_train[:,0])# clf.score(X_test, y_test)
est_Nw = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=0, loss='ls')\
    .fit(X_train, y_train[:,1])# clf.score(X_test, y_test)
est_mu = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls')\
    .fit(X_train, y_train[:,2])# clf.score(X_test, y_test)

In [ ]:
print(MSE(y_test[:,0], est_D0.predict(X_test)))
print(MSE(y_test[:,1], est_Nw.predict(X_test)))
print(MSE(y_test[:,2], est_mu.predict(X_test)))

Exercise if time:
1. What can we change here? 
2. What was different about this training? 
